In [10]:
from ctc_utils.functions import def_gen_expr_for_datasets, def_labels_for_datasets, load_data, unsupervisedClusteringFaiss, loopForDLR
from sklearn.feature_selection import VarianceThreshold
from classifiers.StackedClassifiers import stackedClassifier
from classifiers.DTC import dtc
from classifiers.LogisticRegression import logreg
from classifiers.LinearSVC import lsvc
from classifiers.kNN import knn
from classifiers.MLP import mlp
from classifiers.NB import gnb
from classifiers.SDGC import sdg
from classifiers.hierarchical_classifier import hierarchical_classifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import faiss

In [8]:
# Load data
cell_names, gene_names, labels, gene_expr_bin = load_data()

# Gene expression datasts
# [ge_10xv2, ge_SM2, ge_10xv3, ge_CL, ge_DR, ge_iD, ge_SW, ge_10xv2_2]
gene_expression_datasets = def_gen_expr_for_datasets(gene_expr_bin)

# Labels per dataset
# [lb_10xv2, lb_SM2, lb_10xv3, lb_CL, lb_DR, lb_iD, lb_SW, lb_10xv2_2]
labels_for_datasets = def_labels_for_datasets(labels)

acc_matrix = np.zeros(shape = (8,9))

all_matrix = np.zeros(shape = (8,8,9))

In [11]:
for i, train_dataset in enumerate(gene_expression_datasets):
    print("outer loop :", i)
    # Feature selection
    sel = VarianceThreshold(threshold=0.15)
    train_dataset = sel.fit_transform(train_dataset)

    # Unsupervised clustering using faiss (Euclidean distance)
    index = faiss.IndexFlatL2(len(train_dataset[0])) 
    
    # Setup for the training dataset
    train_dataset = np.ascontiguousarray(train_dataset)
    train_dataset = train_dataset.astype('float32')
    #train model and add cell samples from training dataset
    index.train(train_dataset)
    index.add(train_dataset)
    #calculate neighbors on training dataset
    distances, neighbors_index = index.search(train_dataset, 16)

     # calculate DLR for each cell sample for training of the model.
    dlr_matrix = loopForDLR(neighbors_index, train_dataset)
    print("DLR loop for train data done...")

    for j, test_dataset in enumerate(gene_expression_datasets):
        print("inner loop :", j)

        # Feature selection applied to test dataset
        test_dataset = sel.transform(test_dataset)

        # setup for the testing dataset 
        test_dataset = np.ascontiguousarray(test_dataset)
        test_dataset = test_dataset.astype('float32')
        #Find the neighbors for each cell in the test dataset (neighbors in the train dataset)
        distances_test, neighbors_index_test = index.search(test_dataset, 16)

        # calculate DLR for each cell sample for testing the  model.
        dlr_matrix_test = loopForDLR(neighbors_index_test, train_dataset)
        print("DLR loop for test data  done...")

        print("Define  test & train set")
        if i != j:
            # Define training set
            x_train = dlr_matrix
            y_train = labels_for_datasets[i]

            # Define test set
            x_test = dlr_matrix_test
            y_test = labels_for_datasets[j]

        else: 
            x_train, x_test, y_train, y_test = train_test_split(
            dlr_matrix, labels_for_datasets[j], test_size=0.2, random_state=42
            )

        print("classification...")    

        #Apply knn
        model, acc_knn = knn(x_train, y_train, x_test,y_test)

        #Apply logistic regression
        model, acc_logreg = logreg(x_train, y_train, x_test, y_test)

        #Apply Decision Tree Classifier
        model, acc_dtc = dtc(x_train, y_train, x_test, y_test)

        model, acc_lsvc = lsvc(x_train, y_train, x_test, y_test)

        model, acc_mlp = mlp(x_train, y_train, x_test, y_test)

        model, acc_nb = gnb(x_train, y_train, x_test, y_test)

        model, acc_gnb = gnb(x_train, y_train, x_test, y_test)

        model, acc_sdg = sdg(x_train, y_train, x_test, y_test)

        model, acc_hc = hierarchical_classifier(x_train, y_train, x_test, y_test)

        acc_stacked = stackedClassifier(x_train, y_train, x_test, y_test)


        all_acc_within_dataset = np.array([acc_knn, acc_logreg, acc_dtc, acc_lsvc, acc_mlp, acc_gnb, acc_sdg, acc_hc, acc_stacked])

        print(all_acc_within_dataset)
        acc_matrix[j] = all_acc_within_dataset
    
    all_matrix[i] = acc_matrix
    print(all_matrix)

outer loop : 0
DLR loop for train data  done...
inner loop : 0
DLR loop for test data  done...
Define  test & train set
classification...
[Pipeline] ............. (step 1 of 2) Processing trsvd, total=   0.1s
[Pipeline] ................. (step 2 of 2) Processing m, total=   0.0s


/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[Pipeline] ................ (step 1 of 2) Processing fa, total=   1.6s


/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[Pipeline] ................. (step 2 of 2) Processing m, total=   3.6s


...

/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[Pipeline] ................. (step 2 of 2) Processing m, total=   0.7s


/usr/local/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/usr/local/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

[0.90862791 0.88731687 0.8775979  0.89828709 0.84863852 0.59678956
 0.91736219 0.86920935 0.89454357]
[[[8.95872420e-01 8.84652922e-01 8.60512294e-01 8.89392602e-01
   8.41334615e-01 7.32963052e-01 8.89897310e-01 8.67982332e-01
   8.96111332e-01]
  [6.81394451e-01 6.25833711e-01 6.28077505e-01 6.32064330e-01
   6.12670838e-01 4.94117222e-01 5.96126755e-01 6.20297405e-01
   6.39441703e-01]
  [7.97060388e-01 8.10700517e-01 7.07520857e-01 7.97317255e-01
   7.71250748e-01 6.43753839e-01 7.27401233e-01 7.66458369e-01
   8.57374205e-01]
  [7.24955326e-01 6.76197842e-01 5.89123389e-01 6.22110157e-01
   5.78902419e-01 4.30370239e-01 6.22172643e-01 6.06449013e-01
   6.34845001e-01]
  [2.62221239e-01 3.62835794e-01 2.64010455e-01 5.47634124e-01
   5.08061631e-01 7.62256583e-02 3.05565520e-01 3.75819276e-01
   4.64855032e-01]
  [2.78230459e-01 2.60226604e-01 3.51457244e-01 3.71445512e-01
   4.44619307e-01 1.87706358e-01 2.50480459e-01 3.07250034e-01
   3.01941142e-01]
  [1.66658375e-01 1.87995490

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Save performance results to disk

In [7]:
np.save("../results/performance_matrices/ctc_dlr_res", all_matrix)

# Load performance results from disk

In [1]:
import numpy as np
all_matrix = np.load("../results/performance_matrices/ctc_dlr_res.npy")

# Plotting

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib qt


fig, (pl_10xv2, pl_SM2, pl_10xv3, pl_CL, pl_DR, pl_iD, pl_SW, pl_10xv2_2) = plt.subplots(1, 8, sharey=True)

im = pl_10xv2.imshow(all_matrix[0])
im = pl_SM2.imshow(all_matrix[1])
im = pl_10xv3.imshow(all_matrix[2])
im = pl_CL.imshow(all_matrix[3])
im = pl_DR.imshow(all_matrix[4])
im = pl_iD.imshow(all_matrix[5])
im = pl_SW.imshow(all_matrix[6])
im = pl_10xv2_2.imshow(all_matrix[7])

y_labels = ["10xv2", "SM2", "10xv3", "CL", "DR", "iD", "SW", "10xv2_2"]
x_labels = ["knn", "Logistic Regression", "DTC", "LinearSVC", "MLP", "GaussianNB", "SDG", "Hierarchical", "Stacking" ]

pl_10xv2.set_yticks(np.arange(len(y_labels)))
pl_10xv2.set_yticklabels(y_labels, fontsize = 5)

pl_SM2.set_yticks(np.arange(len(y_labels)))
pl_SM2.set_yticklabels(y_labels, fontsize = 5)

pl_10xv3.set_yticks(np.arange(len(y_labels)))
pl_10xv3.set_yticklabels(y_labels, fontsize = 5)

pl_CL.set_yticks(np.arange(len(y_labels)))
pl_CL.set_yticklabels(y_labels, fontsize = 5)

pl_DR.set_yticks(np.arange(len(y_labels)))
pl_DR.set_yticklabels(y_labels, fontsize = 5)

pl_iD.set_yticks(np.arange(len(y_labels)))
pl_iD.set_yticklabels(y_labels, fontsize = 5)

pl_SW.set_yticks(np.arange(len(y_labels)))
pl_SW.set_yticklabels(y_labels, fontsize = 5)

pl_10xv2_2.set_yticks(np.arange(len(y_labels)))
pl_10xv2_2.set_yticklabels(y_labels, fontsize = 5)


pl_10xv2.set_title("10xv2", fontsize = 6)
pl_10xv2.set_xticks(np.arange(len(x_labels)))
pl_10xv2.set_xticklabels(x_labels, fontsize = 5)
plt.setp(pl_10xv2.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")

pl_SM2.set_title('SM2', fontsize = 6)
pl_SM2.set_xticks(np.arange(len(x_labels)))
pl_SM2.set_xticklabels(x_labels, fontsize = 5)
plt.setp(pl_SM2.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")


pl_10xv3.set_title('10xv3', fontsize = 6)
pl_10xv3.set_xticks(np.arange(len(x_labels)))
pl_10xv3.set_xticklabels(x_labels, fontsize = 5)
plt.setp(pl_10xv3.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")

pl_CL.set_title('CL', fontsize = 6)
pl_CL.set_xticks(np.arange(len(x_labels)))
pl_CL.set_xticklabels(x_labels, fontsize = 5)
plt.setp(pl_CL.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")

pl_DR.set_title('DR', fontsize = 6)
pl_DR.set_xticks(np.arange(len(x_labels)))
pl_DR.set_xticklabels(x_labels, fontsize = 5)
plt.setp(pl_DR.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")


pl_iD.set_title('iD', fontsize = 6)
pl_iD.set_xticks(np.arange(len(x_labels)))
pl_iD.set_xticklabels(x_labels, fontsize = 5)
plt.setp(pl_iD.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")

pl_SW.set_title('SW', fontsize = 6)
pl_SW.set_xticks(np.arange(len(x_labels)))
pl_SW.set_xticklabels(x_labels, fontsize = 5)
plt.setp(pl_SW.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")

pl_10xv2_2.set_title('10xv2_2', fontsize = 6)
pl_10xv2_2.set_xticks(np.arange(len(x_labels)))
pl_10xv2_2.set_xticklabels(x_labels, fontsize = 5)
plt.setp(pl_10xv2_2.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor", fontsize = 5)

fig1 = plt.gcf()

fig.subplots_adjust(right=0.85, top= 1)
cbar_ax = fig.add_axes([0.9, 0.47, 0.01, 0.16])
cbar_ax.axes.tick_params(labelsize = 3.5)
fig1.colorbar(im, cax=cbar_ax)
cbar_ax.set_ylabel('weighted f1 score',fontsize =4)
fig1.text(0.47,0.67, "Training set", ha='center', fontsize= 6)
fig1.text(0.03, 0.55, "Test set", va='center', rotation='vertical', fontsize= 6)



fig, (pl_10xv2, pl_SM2, pl_10xv3, pl_CL, pl_DR, pl_iD, pl_SW, pl_10xv2_2) = plt.subplots(1, 8, sharey=True)
fig1.savefig("../results/report_pdf_results/report_ctc_dlr.pdf", format='pdf', bbox_inches='tight')

plt.show()